In [ ]:
import numpy as np
from utils import get_data, accuracy, save_Yte, get_sweep_id
from kernels import GaussianKernel, LinearKernel
from tqdm import tqdm
import wandb
import math
from sklearn.model_selection import train_test_split 


## Creating model

In [ ]:
class Model:

    def __init__(self, kernel):
        pass

    def fit(self, X, y):
        pass
    
    def predict(self, x):
        pass

## Hyperparameter search

In [ ]:
Xtr, Xte, Ytr = get_data()
Xtr_, Xval_, Ytr_, Yval_ = train_test_split(Xtr, Ytr, test_size=0.1, random_state=123)

In [ ]:
def one_run():
    """Perform one run. Will be used with get_sweep_id."""
    wandb.init(
        project='kernel-challenge',  # nom du projet
        entity='theodumont',         # mon compte
        tags=["Model name"],                # nom du modèle
    )
    # on récupère les paramètres
    sigma = wandb.config.sigma
    lambd = wandb.config.lambd
    # on crée le modèle
    model = Model(kernel=GaussianKernel(sigma=sigma).kernel, lambd=lambd)
    # on le fit
    model.fit(Xtr_, Ytr_)
    # on évalue son accuracy et on la store dans le run sur wandb
    wandb.run.summary["accuracy"] = accuracy(Yval_, model.predict(Xval_))
    
# spécifier le nom range des hyperparamètres, avec 'uniform' ou 'log_uniform
parameters = {
    'sigma': {
        'distribution': 'log_uniform',
        'min': math.log(1e-4),
        'max': math.log(1e2),
    },
}

sweep_id = get_sweep_id(parameters)
%env WANDB_SILENT=False

wandb.agent(sweep_id, function=one_run)

## Making a submission

In [ ]:
model = Model(params)
model.fit(Xtr, Ytr)
Yte = model.predict(Xte)
save_Yte(Yte, model_name="model_name")